# Sparse matrix multiplication

In [ ]:
SELECT a.row_num, b.col_num, sum(a.value*b.value)
FROM a,b
WHERE a.col_num = b.row_num
GROUP BY a.row_num, b.col_num

One element of A*B

In [ ]:
SELECT sum(a.value*b.value)
FROM a,b
WHERE a.col_num = b.row_num
GROUP BY a.row_num, b.col_num
HAVING a.row_num = 2 and b.col_num = 3

# Term-document matrix

Uses reuters.db database file

## Compute similarity matrix

In [ ]:
SELECT f1.docid, f2.docid, sum(f1.count*f2.count)
FROM Frequency f1 JOIN Frequency f2  -- join 2 instances of D matrix to calculate D.D^T
                ON f1.term = f2.term
WHERE f1.docid <= f2.docid           -- calculate lower half only of symetric matrix
GROUP BY f1.docid, f2.docid

This returns a 2561190 x 2561190 document-document square matrix, where each term is the dot product of document vectors. Query is exucuted in 116557ms (~2 minutes).

## Compute similarity of 2 documents

Returns the dot product of 2 document vectors

In [ ]:
SELECT sum(f1.count*f2.count)
    FROM frequency f1 JOIN frequency f2
                      ON f1.term = f2.term
WHERE f1.docid='10080_txt_crude' 
AND f2.docid='17035_txt_earn'

# Find best matching documents to keyword query

Assume query is "washington taxes treasury"

### 1. Create a new document containing the query

In [ ]:
SELECT 'query_doc' as docid, 'washington' as term, 1 as count 
    UNION
SELECT 'query_doc' as docid, 'taxes' as term, 1 as count
    UNION 
SELECT 'query_doc' as docid, 'treasury' as term, 1 as count

### 2. Compute similarity of query document with documents present in the database

In [ ]:
SELECT f1.docid, sum(f1.count*query_table.count) as simil
    from Frequency f1 
                    JOIN 
                        (
                        SELECT 'query_doc' as docid, 'washington' as term, 1 as count 
                            UNION
                        SELECT 'query_doc' as docid, 'taxes' as term, 1 as count
                            UNION 
                        SELECT 'query_doc' as docid, 'treasury' as term, 1 as count
                        ) 
                        query_table
                    ON
                        f1.term = query_table.term
GROUP BY f1.docid
ORDER BY sum(f1.count*query_table.count) desc